# 第六章 角点检测  
## 6.2 Harris角点检测算法  
- 检测步骤：(1)对图像的每个位置，计算窗口移动前后其内部的像素值变化量 (2)计算像素值变化量对应的角点响应函数，并对该函数进行阈值处理，提取角点  
### 6.2.1 计算像素值变化量  
#### 1. 像素值变化量定义
在图像 $I(x,y)$ 的局部窗口$W$内，位移 $(u,v)$ 引起的变化量：
$$
E(u, v) = \sum_{(x,y)\in W} [I(x+u,y+v)-I(x,y)]^2
$$
- $I_x = \partial I/\partial x$, $I_y = \partial I/\partial y$: 图像梯度

#### 2. 泰勒展开近似
一阶泰勒展开：
$$
I(x+u,y+v) \approx I(x,y) + uI_x + vI_y
$$
代入后得到：
$$
E(u,v) \approx \sum_{(x,y)\in W}(uI_x + vI_y)^2
$$

#### 3. 二次型矩阵表示
展开为矩阵形式：
$$
E(u,v) \approx \begin{bmatrix}u & v\end{bmatrix}
\underbrace{
\sum_{(x,y)\in W}
\begin{bmatrix}
\ I_x^2 & I_xI_y \\ 
\ I_xI_y & I_y^2
\end{bmatrix}
}_{M}
\begin{bmatrix}u \\ v\end{bmatrix}
$$
其中 $M =\sum_{(x,y)\in W} H$, H是图像的黑塞矩阵  
ps:黑塞矩阵（Hessian Matrix），又译作海森矩阵、海瑟矩阵、海塞矩阵等，是一个多元函数的二阶偏导数构成的方阵，描述了函数的局部曲率。

#### 4. 椭圆表示的数学基础
##### 特征分解
由于 $M$ 是实对称矩阵，则一定可对角化为：
$$
M = R\Lambda R^T
$$
- $\Lambda = diag(\lambda_1,\lambda_2)$: 特征值矩阵
- $R$: 特征向量矩阵（旋转矩阵），也是正交矩阵，$R^\top = R^{-1}$  

将上式代入$E(u,v)$的计算，得到：  
$$
E(u,v) = \begin{bmatrix}u & v\end{bmatrix} R
\begin{bmatrix}
\ λ_{1} & 0 \\ 
\ 0 & λ_{2} 
\end{bmatrix}
R^\top
\begin{bmatrix}u \\ v\end{bmatrix}
$$  
##### 标准椭圆方程  
令 $\begin{bmatrix}m \ n\end{bmatrix} = \begin{bmatrix}u \ v\end{bmatrix} R$，则：
$$
E(u,v) = m^2\lambda_1 + n^2\lambda_2
$$

#### 5. 几何解释
| 特征值情况 | 几何形状 | 图像特征 |
|------------|----------|----------|
| $\lambda_1 \approx \lambda_2$ 且大 | 圆形 | 角点 | 
| $\lambda_1 \gg \lambda_2$ | 细长椭圆 | 边缘 |
| $\lambda_1 \approx \lambda_2 \approx 0$ | 点 | 平坦区域 |  

### 6.2.2 计算角点响应函数
角点附近的像素值变化量$E[u,v]$对于任意的微小位移$[u,v]$都有很大的值。也就是说，如果一个点是角点，那么它对应的像素值变化量$E[u,v]$的极小值也很大，问题转变为求极小值。将$E[u,v]$表示为两个特征值$\lambda_1,\lambda_2$的函数。我们又知道$[m,n]=[u,v]\mathbf{R}$，那么

$$m^2+n^2=[m,n]\begin{bmatrix}m\\n \end{bmatrix}=[u,v]\mathbf{R}\mathbf{R}^{\mathsf{T}}\begin{bmatrix}u\\v\end{bmatrix}=u^2+v^2 \approx 1$$

假设微小位移$[u,v]$是一个近似单位向量。既然有$m^2+n^2 \approx 1$，根据Rayleighquetient瑞利商的性质，得$E[u,v]$的极大值和极小值分别为

$$\max_{u,v}E[u,v]=\max(\lambda_1,\lambda_2)$$

$$\min_{u,v}E[u,v]=\min(\lambda_1,\lambda_2)$$ 

那么对于一个角点，它对应的两个特征值$\lambda_1,\lambda_2$都应该比较大，因为角点对应的像素值变化量$E[u,v]$的极小值也很大。对于边缘和平坦区域，同样也可以依据两个特征值$\lambda_1,\lambda_2$的大小来确定。总结如下：  
1. 两个特征值都趋于0，对应图像中的平坦区域；
2. 一个特征值大，另一个特征值趋于0，对应图像中的边缘；
3. 两个特征值都大，对应图像中的角点

由此，Harris提出的响应函数为：  
$$\theta = \lambda_1\lambda_2 - \alpha(\lambda_1+\lambda_2)^2$$  
其中$\alpha$是一个常数，通常取0.04-0.06。总结如下：
1. 如果$|\theta|$很小，趋于零，那么对应图像中的平坦区域；
2. 如果$\theta$为负数，那么对应图像中的边缘；
3. 如果$\theta$为很大的正数，那么对应图像中角点。

![响应函数与角点关系](image.png)  

根据线性代数的知识，
$${\rm det}(\mathbf{M}) = \lambda_1\lambda_2$$  
$${\rm trace}(\mathbf{M}) = \lambda_1+\lambda_2$$  
所以可以直接通过矩阵$\mathbf{M}$的行列式和迹来计算角点响应函数，而避免做特征值分解：

$$\theta = \lambda_1\lambda_2 - \alpha(\lambda_1+\lambda_2)^2 = {\rm det}(\mathbf{M}) - \alpha {\rm trace}(\mathbf{M})^2$$



### 6.2.3 Harris角点检测的具体步骤  
(1)计算图像中每个像素的梯度幅值$I_{x}$、$I_{y}$  
(2)将一个局部窗口$W$滑动到图像上每个像素的位置，计算窗口内图像黑塞矩阵的求和矩阵$M=\sum_{(i,j)\in W}H$  
(3)计算角点响应函数$\theta = \lambda_1\lambda_2 - \alpha(\lambda_1+\lambda_2)^2 = {\rm det}(\mathbf{M}) - \alpha {\rm trace}(\mathbf{M})^2$  
(4)对整幅图像的角点响应函数做非极大值抑制    
(5)设定阈值$\gamma$，若$\theta>\gamma$，则为角点  

## 6.3 Harris角点检测代码实现  


In [ ]:
"""""
@Author     :   jiguotong
@Contact    :   1776220977@qq.com
@site       :   
-----------------------------------------------
@Time       :   2025/5/16
@Description:   对图像应用Harris角点检测——自己实现
"""""
from utils import *
from PIL import Image
# 导入一张图像
img = Image.open('sudoku.png')

# 将其转为灰度图像
img_gray = img.convert('L')
img_gray = np.array(img_gray)
img = np.array(img)
corners = harris(img_gray, window=(3,3), threshold=0.002)

plt.imshow(img)
plt.axis('off')
plt.title('source')
plt.show()

img[corners==1] = [255,0,0,255]
plt.imshow(img)
plt.axis('off')
plt.title('harris')
plt.show()

In [ ]:
"""""
@Author     :   jiguotong
@Contact    :   1776220977@qq.com
@site       :   
-----------------------------------------------
@Time       :   2025/5/16
@Description:   第6章 对图像应用Harris角点检测——官方实现
"""""    
img = cv2.imread('sudoku.png')

plt.imshow(img[:, :, ::-1])
plt.axis('off')
plt.title('Original Input')
plt.show()

response = harris_corners(img)
img[response == 255] = [0, 0, 255]
plt.imshow(img[:, :, ::-1])
plt.axis('off')
plt.title('Harris Corner Response')
plt.show()